## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'
fileName = 'jet.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (15,5))

img = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)
hist = cv2.calcHist([img], [0], None, [256], [0, 256])

axs[0].imshow(img, cmap='gray')
axs[0].set_title('Obraz')
axs[0].axis("off")

axs[1].plot(hist)
axs[1].set_xlim([0, 256])
axs[1].set_title('Histogram')
axs[1].grid()

2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).

In [ ]:
lm = np.argmax(hist)


3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.wD
Następnie znaleźć minimum - `np.argmin`.

In [ ]:
avg_brightness = img.mean()
print(avg_brightness)

cumulative_hist = np.cumsum(hist)
half_cumulative = np.abs(cumulative_hist - (img.size / 2))
dsihe = np.argmin(half_cumulative)

division_point = avg_brightness
division_point_2 = dsihe



4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.

In [ ]:
H1 = hist[:lm]
H2 = hist[lm:]

C1 = np.cumsum(H1)
C2 = np.cumsum(H2)

C1_normalized = C1 / np.max(C1)
C2_normalized = C2 / np.max(C2)

5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.

In [ ]:
C1n = division_point * C1_normalized
C2n = division_point + 1 + (255 - division_point - 1) * C2_normalized

Cn = np.zeros(256)
Cn[:lm] = C1n
Cn[lm:] = C2n


C1n_2 = division_point_2 * C1_normalized
C2n_2 = division_point_2 + 1 + (255 - division_point_2 - 1) * C2_normalized

Cn_2 = np.zeros(256)
Cn_2[:lm] = C1n_2
Cn_2[lm:] = C2n_2

6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
img_bbhe = cv2.LUT(img, Cn.astype('uint8'))

img_dsihe = cv2.LUT(img, Cn_2.astype('uint8'))

img_he = cv2.equalizeHist(img)

fig, axs = plt.subplots(1, 3, figsize=(25, 30))

axs[0].imshow(img_bbhe, cmap='gray')
axs[0].set_title('BBHE')
axs[0].axis("off")

axs[1].imshow(img_dsihe, cmap='gray')
axs[1].set_title('DSIHE')
axs[1].axis("off")

axs[2].imshow(img_he, cmap='gray')
axs[2].set_title('HE')
axs[2].axis("off")

plt.show()
